In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

p_root = "/home/colombelli/Documents/datasets/LUNG_TEST_NPN_TRUNC/"
df0_name = "tcga_intersect.csv"
df1_name = "cumida1_intersect.csv"
df2_name = "cumida2_intersect.csv"
df3_name = "cumida3_intersect.csv"

In [3]:
df0 = pd.read_csv(p_root+df0_name, index_col=0)
df1 = pd.read_csv(p_root+df1_name, index_col=0)
df2 = pd.read_csv(p_root+df2_name, index_col=0)
df3 = pd.read_csv(p_root+df3_name, index_col=0)

In [5]:
def normalize_df(df):
    
    df_norm_vals = normalize(df.iloc[:, 0:-1])
    cols =  df.iloc[:, 0:-1].columns
    indexes = df.iloc[:, 0:-1].index
    
    df_norm = pd.DataFrame(df_norm_vals)
    df_norm.columns = cols
    df_norm.index = indexes
    df_norm['class'] = df['class']
    
    return df_norm

In [7]:
df0_norm = normalize_df(df0)
scaler = StandardScaler().fit(df0_norm.iloc[:, 0:-1])

In [8]:
def scale_df(df, save_path):
    
    df_scaled_vals = scaler.transform(df.iloc[:, 0:-1])
    cols =  df.iloc[:, 0:-1].columns
    indexes = df.iloc[:, 0:-1].index
    
    df_scaled = pd.DataFrame(df_scaled_vals)
    df_scaled.columns = cols
    df_scaled.index = indexes
    df_scaled['class'] = df['class']
    
    df_scaled.to_csv(save_path)
    print("Scaling completed for:", save_path)
    return

In [9]:
dfs = [df0,df1,df2,df3]
paths = [p_root+"tcga_scaled_and_normalized.csv",
         p_root+"cumida1_scaled_and_normalized.csv",
         p_root+"cumida2_scaled_and_normalized.csv",
         p_root+"cumida3_scaled_and_normalized.csv"]

In [10]:
for i, df in enumerate(dfs):
    df_n = normalize_df(df)
    scale_df(df_n, paths[i])

Scaling completed for: /home/colombelli/Documents/datasets/LUNG_TEST_NPN_TRUNC/tcga_scaled_and_normalized.csv
Scaling completed for: /home/colombelli/Documents/datasets/LUNG_TEST_NPN_TRUNC/cumida1_scaled_and_normalized.csv
Scaling completed for: /home/colombelli/Documents/datasets/LUNG_TEST_NPN_TRUNC/cumida2_scaled_and_normalized.csv
Scaling completed for: /home/colombelli/Documents/datasets/LUNG_TEST_NPN_TRUNC/cumida3_scaled_and_normalized.csv


## Box-plotting norm-scaled data

In [11]:
import seaborn as sns

def boxplot_df(df):
    x = sns.boxplot(data=df)
    x.set(xticklabels=[])
    x.tick_params(bottom=False)
    x.set(xlabel=None)
    return